<a href="https://colab.research.google.com/github/b-fatma/S2I-DM/blob/master/src/5_supervised/random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install scikit-optimize
!pip install imbalanced-learn

# Loading the data

In [ ]:
y_train = pd.read_csv('/content/drive/MyDrive/dm_fire_prediction/feature_engineering/v1/y_train.csv')
y_test = pd.read_csv('/content/drive/MyDrive/dm_fire_prediction/feature_engineering/v1/y_test.csv')

X_test = pd.read_csv('/content/drive/MyDrive/dm_fire_prediction/feature_engineering/v1/X_test.csv')
X_train = pd.read_csv('/content/drive/MyDrive/dm_fire_prediction/feature_engineering/v1/X_train.csv')

X_test_scaled = pd.read_csv('/content/drive/MyDrive/dm_fire_prediction/feature_engineering/v1/X_test_scaled.csv')
X_train_scaled = pd.read_csv('/content/drive/MyDrive/dm_fire_prediction/feature_engineering/v1/X_train_scaled.csv')

In [ ]:
y_train = y_train.squeeze()
y_test = y_test.squeeze()

# Random Forest

In [ ]:
from skopt import BayesSearchCV
from skopt.space import Integer, Categorical, Real

rf = RandomForestClassifier(random_state=0, n_jobs=-1)

search_space = {
    'n_estimators': Integer(100, 300),
    'max_depth': Integer(10, 30),
    'min_samples_split': Integer(2, 10),
    'min_samples_leaf': Integer(1, 5),
    'max_features': Categorical(['sqrt', 'log2']),
    'class_weight': Categorical([None, 'balanced'])
}

bayes_search = BayesSearchCV(
    estimator=rf,
    search_spaces=search_space,
    n_iter=30,
    cv=5,
    scoring='f1',
    n_jobs=-1,
    random_state=0,
    verbose=1
)

bayes_search.fit(X_train, y_train)

best_model = bayes_search.best_estimator_
print(bayes_search.best_params_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

In [ ]:
print(classification_report(y_train, best_model.predict(X_train)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     38228
           1       0.98      1.00      0.99     11469

    accuracy                           1.00     49697
   macro avg       0.99      1.00      0.99     49697
weighted avg       1.00      1.00      1.00     49697



# Evaluation on the test set

In [ ]:
print(classification_report(y_test, best_model.predict(X_test)))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      4248
           1       0.90      0.92      0.91      1274

    accuracy                           0.96      5522
   macro avg       0.94      0.94      0.94      5522
weighted avg       0.96      0.96      0.96      5522



# From scratch

In [ ]:
import numpy as np
from collections import Counter

class RandomForest:
    def __init__(self, n_estimators=100, max_depth=10, min_samples_split=2,
                 max_features='sqrt', bootstrap=True, random_state=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.bootstrap = bootstrap
        self.random_state = random_state
        self.trees = []

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)

        if self.random_state is not None:
            np.random.seed(self.random_state)

        self.trees = []
        n_samples = X.shape[0]

        for i in range(self.n_estimators):
            tree = DecisionTree(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                max_features=self._get_max_features(X.shape[1])
            )

            if self.bootstrap:
                indices = np.random.choice(n_samples, n_samples, replace=True)
                X_sample = X[indices]
                y_sample = y[indices]
            else:
                X_sample = X
                y_sample = y

            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

        return self

    def predict(self, X):
        X = np.array(X)

        tree_predictions = np.array([tree.predict(X) for tree in self.trees])
        tree_predictions = np.swapaxes(tree_predictions, 0, 1)

        predictions = [self._majority_vote(pred) for pred in tree_predictions]
        return np.array(predictions)

    def _majority_vote(self, predictions):
        return Counter(predictions).most_common(1)[0][0]

    def _get_max_features(self, n_features):
        if self.max_features == 'sqrt':
            return int(np.sqrt(n_features))
        elif self.max_features == 'log2':
            return int(np.log2(n_features))
        elif isinstance(self.max_features, int):
            return self.max_features
        elif isinstance(self.max_features, float):
            return int(self.max_features * n_features)
        else:
            return n_features


class DecisionTreeNode:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None


class DecisionTree:
    def __init__(self, max_depth=10, min_samples_split=2, max_features=None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.root = None

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        self.n_features = X.shape[1]
        self.root = self._build_tree(X, y, depth=0)
        return self

    def predict(self, X):
        if self.root is None:
            raise ValueError("Model not fitted. Call fit() first.")
        X = np.array(X)
        return np.array([self._predict_single(row, self.root) for row in X])

    def _build_tree(self, X, y, depth):
        num_samples, num_features = X.shape

        if (
            depth >= self.max_depth
            or len(np.unique(y)) == 1
            or num_samples < self.min_samples_split
        ):
            return DecisionTreeNode(value=self._most_common_label(y))

        feature, threshold = self._best_split(X, y)

        if feature is None:
            return DecisionTreeNode(value=self._most_common_label(y))

        left_idx = X[:, feature] <= threshold
        right_idx = X[:, feature] > threshold

        if np.sum(left_idx) == 0 or np.sum(right_idx) == 0:
            return DecisionTreeNode(value=self._most_common_label(y))

        left_child = self._build_tree(X[left_idx], y[left_idx], depth + 1)
        right_child = self._build_tree(X[right_idx], y[right_idx], depth + 1)

        return DecisionTreeNode(
            feature=feature,
            threshold=threshold,
            left=left_child,
            right=right_child,
        )

    def _best_split(self, X, y):
        best_gain = -1
        best_feature = None
        best_threshold = None

        n_samples, n_features = X.shape

        if self.max_features is not None:
            features = np.random.choice(n_features, self.max_features, replace=False)
        else:
            features = range(n_features)

        for feature in features:
            feature_values = X[:, feature]
            sorted_vals = np.unique(feature_values)

            if len(sorted_vals) < 2:
                continue

            thresholds = (sorted_vals[:-1] + sorted_vals[1:]) / 2

            for threshold in thresholds:
                left_idx = feature_values <= threshold
                right_idx = feature_values > threshold

                if np.sum(left_idx) == 0 or np.sum(right_idx) == 0:
                    continue

                gain = self._information_gain(y, feature_values, threshold)

                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold

        return best_feature, best_threshold

    def _entropy(self, y):
        if len(y) == 0:
            return 0
        counts = Counter(y)
        probabilities = [count / len(y) for count in counts.values()]
        return -np.sum([p * np.log2(p) for p in probabilities if p > 0])

    def _information_gain(self, y, feature_values, threshold):
        parent_entropy = self._entropy(y)

        left_idx = feature_values <= threshold
        right_idx = feature_values > threshold

        if np.sum(left_idx) == 0 or np.sum(right_idx) == 0:
            return 0

        n = len(y)
        n_left = np.sum(left_idx)
        n_right = np.sum(right_idx)

        child_entropy = (n_left / n) * self._entropy(y[left_idx]) + \
                        (n_right / n) * self._entropy(y[right_idx])

        return parent_entropy - child_entropy

    def _predict_single(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._predict_single(x, node.left)
        else:
            return self._predict_single(x, node.right)

    def _most_common_label(self, y):
        if len(y) == 0:
            return None
        return Counter(y).most_common(1)[0][0]

In [16]:
from sklearn.metrics import classification_report, accuracy_score
import time

rf = RandomForest(n_estimators=100, max_depth=20, random_state=42)

s_time = time.time()
rf.fit(X_train, y_train)
print(f"Training time: {time.time() - s_time:.2f}s")

s_time = time.time()
y_pred = rf.predict(X_test)
print(f"Prediction time: {time.time() - s_time:.2f}s\n")

print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")

Training time: 32661.58s
Prediction time: 2.05s

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      4248
           1       0.90      0.90      0.90      1274

    accuracy                           0.95      5522
   macro avg       0.93      0.93      0.93      5522
weighted avg       0.95      0.95      0.95      5522

Accuracy: 0.9533
